In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
import zipfile
from math import log
from random import shuffle

import nltk
from numpy import mean

zipFile = zipfile.ZipFile("sumarizarea-documentelor.zip")

business_summaries = [f for f in zipFile.namelist() if 'BBC News Summary/Summaries/business/' in f]
entertainment_summaries = [f for f in zipFile.namelist() if 'BBC News Summary/Summaries/entertainment/' in f]
politics_summaries = [f for f in zipFile.namelist() if 'BBC News Summary/Summaries/politics/' in f]
sport_summaries = [f for f in zipFile.namelist() if 'BBC News Summary/Summaries/sport/' in f]
tech_summaries = [f for f in zipFile.namelist() if 'BBC News Summary/Summaries/tech/' in f]

business_files = [f for f in zipFile.namelist() if 'BBC News Summary/News Articles/business/' in f]
entertainment_files = [f for f in zipFile.namelist() if 'BBC News Summary/News Articles/entertainment/' in f]
politics_files = [f for f in zipFile.namelist() if 'BBC News Summary/News Articles/politics/' in f]
sport_files = [f for f in zipFile.namelist() if 'BBC News Summary/News Articles/sport/' in f]
tech_files = [f for f in zipFile.namelist() if 'BBC News Summary/News Articles/tech/' in f]

business_summaries = business_summaries[1:]
entertainment_summaries = entertainment_summaries[1:]
politics_summaries = politics_summaries[1:]
sport_summaries = sport_summaries[1:]
tech_summaries = tech_summaries[1:]

business_files = business_files[1:]
entertainment_files = entertainment_files[1:]
politics_files = politics_files[1:]
sport_files = sport_files[1:]
tech_files = tech_files[1:]

In [ ]:
business = []
entertainment = []
politics = []
sport = []
tech = []

for i in range(len(business_files)):
    business.append((business_files[i], business_summaries[i]))

for i in range(len(entertainment_files)):
    entertainment.append((entertainment_files[i], entertainment_summaries[i]))

for i in range(len(politics_files)):
    politics.append((politics_files[i], politics_summaries[i]))

for i in range(len(sport_files)):
    sport.append((sport_files[i], sport_summaries[i]))

for i in range(len(tech_files)):
    tech.append((tech_files[i], tech_summaries[i]))

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from re import sub
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

nltk.download('punkt')
nltk.download('wordnet')

STOP_WORDS = [line.strip() for line in open("stop_words")]

classes = ["business", "entertainment", "politics", "sport", "tech"]
methods_naive_bayes = ['No lemmatisation,\nwithout stop words', 'No lemmatisation,\n with stop words', 'Lemmatisation,\n with stop words']
methods_tf_idf = ["standard\n", "similarity with title", "weighting sentences, \n similarity with title"]
methods_complete = ["Naive Bayes, 'No lemmatisation, without stop words', unigrams", "Naive Bayes, 'No lemmatisation, with stop words', unigrams",
                   "Naive Bayes, 'Lemmatisation, with stop words', unigrams", "Naive Bayes, 'No lemmatisation, without stop words', bigrams",
                   "Naive Bayes, 'No lemmatisation, with stop words', bigrams", "Naive Bayes, 'Lemmatisation, with stop words', bigrams",
                   "Naive Bayes, 'No lemmatisation, without stop words', 4-grams",
                   "Naive Bayes, 'No lemmatisation, with stop words', 4-grams", "Naive Bayes, 'Lemmatisation, with stop words', 4-grams",
                   "Tf-Idf, standard, unigrams", "Tf-Idf, similarity with title, unigrams", 
                   "Tf-Idf, weighting sentences and similarity with title, unigrams", "Tf-Idf, standard, bigrams", "Tf-Idf, similarity with title, bigrams",
                   "Tf-Idf, weighting sentences and similarity with title, bigrams", "Tf-Idf, standard, 4-grams", "Tf-Idf, similarity with title, 4-grams", 
                   "Tf-Idf, weighting sentences and similarity with title, 4-grams"]

UNIGRAMS = 1
BIGRAMS = 2
FOUR_GRAMS = 4
N_GRAMS = [UNIGRAMS, BIGRAMS, FOUR_GRAMS]

In [ ]:
class Files:
    def __init__(self, type, nb_train, files, no_files):
        self.type = type
        self.nb_train = nb_train
        self.files = files
        self.no_files = no_files
        self.get_train_test_files()

    def get_train_test_files(self):
        shuffle(self.files)
        self.train_files = self.files[:self.nb_train]
        self.test_files = self.files[self.nb_train:]


def form_n_grams(data, n_grams):
    n_grams_data = []
    for i in range(len(data) - n_grams + 1):
        n_gram = data[i: i + n_grams]
        n_grams_data.append(tuple(n_gram))
    return n_grams_data


def parse_document(path, stop_words, lemmatise, n_grams, method, replace):
    if replace:
        with open(path, 'r') as file:
            data = file.read().replace("\n", '.', 1)
        #to_replace = "0123456789,/?<>;:\'\"{}[]|\\+=_-)(*&^%$#@!~`"
        #for c in to_replace:
            #data = data.replace(c, '')
    else:
        with open(path, 'r') as file:
            data = file.read()
    wnl = WordNetLemmatizer()

    sentences = nltk.tokenize.sent_tokenize(sub(r'[\.?!]+([A-Z])', r'. \1', data))
    no_stop_words = {}
    for sentence in sentences:
        words = word_tokenize(sentence)
        no_stop_words[sentence] = []
        for word in words:
            if not lemmatise:
                if stop_words:
                    if word not in STOP_WORDS and word not in no_stop_words[sentence]:
                        no_stop_words[sentence].append(word)
                else:
                    if word not in no_stop_words[sentence]:
                        no_stop_words[sentence].append(word)
            else:
                base_word = wnl.lemmatize(word)
                if base_word not in STOP_WORDS and base_word not in no_stop_words[sentence]:
                    no_stop_words[sentence].append(base_word)
    if method == "Naive Bayes":
        for sentence in no_stop_words.keys():
            no_stop_words[sentence] = form_n_grams(no_stop_words[sentence], n_grams)

    return no_stop_words

def get_sentences(path):
    with open(path, 'r') as file:
            data = file.read().replace("\n", ".", 1)
    return nltk.tokenize.sent_tokenize(sub(r'[\.?!]+([A-Z])', r'. \1', data))

In [ ]:
print(get_sentences(business_files[0]))

In [ ]:
def split_training():
    train_business_no = int(0.75 * len(business_summaries))

    train_entertainment_no = int(0.75 * len(entertainment_summaries))

    train_politics_no = int(0.75 * len(politics_summaries))

    train_sport_no = int(0.75 * len(sport_summaries))

    train_tech_no = int(0.75 * len(tech_summaries))

    return [Files("business", train_business_no, business, len(business_files)), Files("entertainment",
                                                                                       train_entertainment_no,
                                                                                       entertainment,
                                                                                       len(entertainment_files)),
            Files("politics", train_politics_no, politics, len(politics_files)), Files("sport",
                                                                                       train_sport_no,
                                                                                       sport,
                                                                                       len(sport_files)),
            Files("tech", train_tech_no, tech, len(tech_files))]


def count_n_grams(sentence, _vocabulary, keep):
    if keep:
        for n_gram in sentence:
            if n_gram not in _vocabulary:
                _vocabulary[n_gram] = (1, 0)
            else:
                _vocabulary[n_gram] = (_vocabulary[n_gram][0] + 1, _vocabulary[n_gram][1])
    else:
        for n_gram in sentence:
            if n_gram not in _vocabulary:
                _vocabulary[n_gram] = (0, 1)
            else:
                _vocabulary[n_gram] = (_vocabulary[n_gram][0], _vocabulary[n_gram][1] + 1)
    return _vocabulary


def count_n_grams_file(file_path, file_summary, _vocabulary, no_sentences, no_n_grams, stop_words, lemmatise, n_grams):
    sentences = parse_document(file_path, stop_words, lemmatise, n_grams, "Naive Bayes", True)
    sentences_to_keep = parse_document(file_summary, stop_words, lemmatise, n_grams, "Naive Bayes", True)
    no_sentences = (no_sentences[0] + len(sentences_to_keep), no_sentences[1] +
                    len(sentences) - len(sentences_to_keep))
    for sentence in sentences:
        if sentence in sentences_to_keep:
            no_n_grams = (no_n_grams[0] + len(sentences[sentence]), no_n_grams[1])
            count_n_grams(sentences[sentence], _vocabulary, True)
        else:
            no_n_grams = (no_n_grams[0], no_n_grams[1] + len(sentences[sentence]))
            count_n_grams(sentences[sentence], _vocabulary, False)
    return (no_sentences, no_n_grams, _vocabulary)


In [ ]:
def print_file(message, path):
    print(message)
    with open(path, 'r') as file:
        data = file.read()
    print(data)

In [ ]:
def train_files(files, stop_words, lemmatise, n_grams):
    _vocabulary = {}
    no_sentences = (0, 0)
    no_n_grams = (0, 0)
    for category in files:
        for (file_article, file_summary) in category.train_files:
            (no_sentences, no_n_grams, _vocabulary) = count_n_grams_file(file_article, file_summary, _vocabulary,
                                                                         no_sentences, no_n_grams, stop_words,
                                                                         lemmatise, n_grams)

    return (no_sentences, no_n_grams, _vocabulary)


def predict(params, sentence, alpha=1):
    (no_sentences, no_n_grams, _vocabulary) = params
    log_keep = log(no_sentences[0] / (no_sentences[0] + no_sentences[1]))
    log_no_keep = log(no_sentences[1] / (no_sentences[0] + no_sentences[1]))
    for n_gram in sentence:
        if n_gram not in _vocabulary:
            _vocabulary[n_gram] = (0, 0)
        log_keep += log((_vocabulary[n_gram][0] + alpha) / (no_n_grams[0] + len(_vocabulary.keys()) * alpha))
        log_no_keep += log((_vocabulary[n_gram][1] + alpha) / (no_n_grams[1] + len(_vocabulary.keys()) * alpha))
    if log_keep > log_no_keep:
        return 'keep'
    return 'no_keep'


def form_sumarization(sentences):
    s = ''
    for sentence in sentences:
        s += sentence
    return s

In [ ]:
def evaluate_model_naive_bayes():
    files = split_training()
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rouge4'], use_stemmer=True)
    tests = [(False, False), (True, False), (True, True)]
    _n_grams_ = {UNIGRAMS: 'unigrame', BIGRAMS: 'bigrame', FOUR_GRAMS:'4-grame'}
    _precisions = {"business unigrams": [], "entertainment unigrams": [], "politics unigrams": [], "sport unigrams": [],
                  "tech unigrams": [], "business bigrams": [], "entertainment bigrams": [], "politics bigrams": [],
                  "sport bigrams": [], "tech bigrams": [], "business 4-grams": [], "entertainment 4-grams": [], "politics 4-grams": [], "sport 4-grams": [],
                  "tech 4-grams": []}
    _recalls = {"business unigrams": [], "entertainment unigrams": [], "politics unigrams": [], "sport unigrams": [],
                  "tech unigrams": [], "business bigrams": [], "entertainment bigrams": [], "politics bigrams": [],
                  "sport bigrams": [], "tech bigrams": [], "business 4-grams": [], "entertainment 4-grams": [], "politics 4-grams": [], "sport 4-grams": [],
                "tech 4-grams": []}
    precisions_complete = {"business":[], "entertainment":[], "politics":[], "sport":[], "tech":[]}
    recalls_complete = {"business":[], "entertainment":[], "politics":[], "sport":[], "tech":[]}
    cantitative_analysis = {"business":[], "entertainment":[], "politics":[], "sport":[], "tech":[]}
    nb_files = 0
    for n_grams in N_GRAMS:
        for test in tests:
            if not test[0]:
                stop_words = 'fara'
            else:
                stop_words = 'cu'
            if not test[1]:
                lemmatise = 'fara'
            else:
                lemmatise = 'cu'
            params = train_files(files, test[0], test[1], n_grams)
            for category in files:
                precisions = []
                recalls = []
                count = 0
                for (file_article, file_summary) in category.train_files:
                    sentences = parse_document(file_article, test[0], test[1], n_grams, "Naive Bayes", True)
                    sentences_to_keep = parse_document(file_summary, test[0], test[1], n_grams, "Naive Bayes", True)
                    sumarization = form_sumarization(sentences_to_keep)
                    unreplaced_sentences = get_sentences(file_article)
                    predicted_sents = ''
                    count += 1
                    i = 0
                    for sentence in sentences:
                        result = predict(params, sentences[sentence])
                        if result == 'keep':
                            predicted_sents += unreplaced_sentences[i]
                        i += 1
                    if count < 3:
                        print_file("Text:(train files, categoria " + category.type + ' ' + stop_words + ' eliminarea cuvintelor comune,' + lemmatise + ' lematizare si folosind ' +  _n_grams_[n_grams] + ")\n", file_article)
                        print_file("Summary:\n", file_summary)
                        print("\nPredicted summarisation:\n\n", predicted_sents)
                    if count == 3:
                        break
                for (file_article, file_summary) in category.test_files:
                    sentences = parse_document(file_article, test[0], test[1], n_grams, "Naive Bayes", True)
                    sentences_to_keep = parse_document(file_summary, test[0], test[1], n_grams, "Naive Bayes", True)
                    sumarization = form_sumarization(sentences_to_keep)
                    unreplaced_sentences = get_sentences(file_article)
                    predicted_sents = ''
                    count += 1
                    i = 0
                    for sentence in sentences:
                        result = predict(params, sentences[sentence])
                        if result == 'keep':
                            predicted_sents += unreplaced_sentences[i]
                        i += 1
                    if count < 5:
                        print_file("Text:(test files, categoria " + category.type + ' ' + stop_words + ' eliminarea cuvintelor comune,' + lemmatise + ' lematizare si folosind ' +  _n_grams_[n_grams] + ")\n", file_article)
                        print_file("Summary:\n", file_summary)
                        print("\nPredicted summarisation:\n\n", predicted_sents)
                            
                    if len(cantitative_analysis[category.type]) < 3:
                            with open(file_article, 'r') as file:
                                data = file.read()
                            cantitative_analysis[category.type] += [(data, predicted_sents, sumarization)]
                
                    scores = scorer.score(sumarization, predicted_sents)
                    if n_grams == UNIGRAMS:
                        if len(predicted_sents.split()) != 0:
                            precisions.append(scores['rouge1'].recall * len(sumarization.split()) / len(predicted_sents.split()))
                        else:
                            precisions.append(0)
                        recalls.append(scores['rouge1'].precision)
                    elif n_grams == BIGRAMS:
                        if len(predicted_sents.split()) != 0:
                            precisions.append(scores['rouge2'].recall * len(sumarization.split()) / len(predicted_sents.split()))
                        else:
                            precisions.append(0)
                        recalls.append(scores['rouge2'].recall)
                    else:
                        if len(predicted_sents.split()) != 0:
                            precisions.append(scores['rouge4'].recall * len(sumarization.split()) / len(predicted_sents.split()))
                        else:
                            precisions.append(0)
                        recalls.append(scores['rouge4'].recall)
                if n_grams == UNIGRAMS:
                    key = category.type + " unigrams"
                elif n_grams == BIGRAMS:
                    key = category.type + " bigrams"
                else:
                    key = category.type + " 4-grams"
                _precisions[key].append(100 * mean(precisions))
                _recalls[key].append(100 * mean(recalls))
                precisions_complete[category.type].append(100 * mean(precisions))
                recalls_complete[category.type].append(100* mean(recalls))
                print('Precizia clasei ', category.type, stop_words, 'eliminarea cuvintelor comune',
                      lemmatise, 'lematizare si folosind', _n_grams_[n_grams], 'este:', 100 * mean(precisions), '%')
                print('Regasirea clasei ', category.type, stop_words, 'eliminarea cuvintelor comune',
                      lemmatise, 'lematizare si folosind', _n_grams_[n_grams], 'este:', 100 * mean(recalls), '%')
            print('====================================================================================================')
    print(cantitative_analysis)
    return (_precisions, _recalls, precisions_complete, recalls_complete, cantitative_analysis)

In [ ]:
(_precisions, _recalls, _precisions_complete, _recalls_complete, cantitative_analysis) = evaluate_model_naive_bayes()

In [ ]:
print(cantitative_analysis)

In [ ]:
for _class in classes:
    for _tuple in cantitative_analysis[_class]:
        print(f'Text:\n{_tuple[0]}\nPredicted sumarization:\n{_tuple[1]}\nSumarization:\n{_tuple[2]}')
        print('======================================================================================================')

In [ ]:
def counstruct_n_grams_nouns(sentence, n_grams):
    sentence = word_tokenize(sentence)
    nouns = []
    for word in sentence:
        ans = nltk.pos_tag([word])
        val = ans[0][1]
        if(val == 'NN' or val == 'NNS' or val == 'NNPS' or val == 'NNP'):
            nouns.append(word)
    nouns_n_grams = []
    for i in range(len(nouns) - n_grams + 1):
        nouns_n_grams.append(nouns[i : i + n_grams])
    return nouns_n_grams

In [ ]:
def construct_vocabulary(documents, n_grams):
    _vocabulary_tf = {}
    _vocabulary_idf = {}
    for (document, _) in documents:
        #print(document)
        sentences = parse_document(document, False, False, n_grams, "Tf-Idf", True)
        for sentence in sentences:
            nouns_n_grams = counstruct_n_grams_nouns(sentence, n_grams)
            for noun_n_gram in nouns_n_grams:
                key = tuple(noun_n_gram) + tuple(document)
                if key not in _vocabulary_tf:
                    _vocabulary_tf[key] = 0
                else:
                    _vocabulary_tf[key] += 1
                key = tuple(noun_n_gram)
                if key not in _vocabulary_idf:
                    _vocabulary_idf[key] = (1, True)
                else:
                    if not _vocabulary_idf[key]:
                        _vocabulary_idf[key] = (vocabulary_idf[key][0] + 1, True)
        for sentence in sentences:
            nouns_n_grams = counstruct_n_grams_nouns(sentence, n_grams)
            for noun_n_gram in nouns_n_grams:
                key = tuple(noun_n_gram)
                _vocabulary_idf[key] = (_vocabulary_idf[key][0], False)
    return (_vocabulary_tf, _vocabulary_idf, len(documents))

In [ ]:
def tf_idf(word_gram, document, params):
    (_vocabulary_tf, _vocabulary_idf, nb_documents) = params
    return _vocabulary_tf[tuple(word_gram) + tuple(document)] * log (nb_documents / _vocabulary_idf[tuple(word_gram)][0])

In [ ]:
def get_title(path):
    with open(path, 'r') as file:
        data = file.read()
        
    return word_tokenize(data[:data.index('\n')])

In [ ]:
def score_sentence_stageI(sentence, document, params, n_grams):
    score = 0.0
    nouns_n_grams = counstruct_n_grams_nouns(sentence, n_grams)
    for noun_n_gram in nouns_n_grams:
        score += tf_idf(noun_n_gram, document, params)
    return score

In [ ]:
def score_sentence_stageII(sentence, document, params, n_grams):
    score = score_sentence_stageI(sentence, document, params, n_grams)
    title = get_title(document)
    count = 0.0
    sentence = word_tokenize(sentence)
    for word in sentence:
        if word in title:
            count += 1
    score += count / len(title)
    return 0.1 * score

In [ ]:
def score_sentence_stageIII(sentence, document, params, n_grams):
    score = score_sentence_stageII(sentence, document, params, n_grams)
    sentences = parse_document(document, False, False, UNIGRAMS,"Tf-Idf", True)
    sentences = list(sentences.keys())
    return score * sentences.index(sentence) / len(sentences)

In [ ]:
def tf_idf_sumarization(document, documents, method, params, n_grams):
    sentences = parse_document(document, False, False, UNIGRAMS,"Tf-Idf", True)
    scores = []
    pairs = []
    sumarization = ''
    for sentence in sentences:
        #print(sentences[sentence])
        score = method(sentence, document, params, n_grams)
        scores.append(score)
        pairs.append((score, sentence))
    mean_score = mean(scores)
    for pair in pairs:
        if pair[0] >= mean_score:
            sumarization += pair[1]
    return sumarization

In [ ]:
def bleu_score(candidate, reference, n_grams):
    candidate_n_grams = form_n_grams(candidate, n_grams)
    reference_n_grams = form_n_grams(reference, n_grams)
    count = 0.0
    for candidate_n_gram in candidate_n_grams:
        if candidate_n_gram in reference_n_grams:
            count += 1
    return count / len(candidate_n_grams)

In [ ]:
def rouge_score(candidate, reference, n_grams):
    candidate_n_grams = form_n_grams(candidate, n_grams)
    reference_n_grams = form_n_grams(reference, n_grams)
    count = 0.0
    for candidate_n_gram in candidate_n_grams:
        if candidate_n_gram in reference_n_grams:
            count += 1
    return count / len(reference_n_grams)

In [ ]:
def evaluate_model_tf_idf():
    files = split_training()
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rouge4'], use_stemmer=True)
    _n_grams_ = {UNIGRAMS: 'unigrame', BIGRAMS: 'bigrame', FOUR_GRAMS:'4-grame'}
    methods = {score_sentence_stageI:"standard",  score_sentence_stageII:"similaritatii cu titlul",  score_sentence_stageIII:"ponderarii propozitiilor si a similaritatii cu titlul"}
    _precisions = {"business unigrams": [], "entertainment unigrams": [], "politics unigrams": [], "sport unigrams": [],
                  "tech unigrams": [], "business bigrams": [], "entertainment bigrams": [], "politics bigrams": [],
                  "sport bigrams": [], "tech bigrams": [], "business 4-grams": [], "entertainment 4-grams": [], "politics 4-grams": [], "sport 4-grams": [],
                  "tech 4-grams": []}
    _recalls = {"business unigrams": [], "entertainment unigrams": [], "politics unigrams": [], "sport unigrams": [],
                  "tech unigrams": [], "business bigrams": [], "entertainment bigrams": [], "politics bigrams": [],
                  "sport bigrams": [], "tech bigrams": [], "business 4-grams": [], "entertainment 4-grams": [], "politics 4-grams": [], "sport 4-grams": [],
                "tech 4-grams": []}
    precisions_complete = {"business":[], "entertainment":[], "politics":[], "sport":[], "tech":[]}
    recalls_complete = {"business":[], "entertainment":[], "politics":[], "sport":[], "tech":[]}
    cantitative_analysis = {"business":[], "entertainment":[], "politics":[], "sport":[], "tech":[]}
    
    for n_grams in N_GRAMS:
        for category in files:
            params = construct_vocabulary(category.files, n_grams)
            for method in methods:
                precisions = []
                recalls = []
                count  = 0
                for (file_article, file_summary) in category.train_files:
                    sentences_to_keep = parse_document(file_summary, False, False, n_grams, "Naive Bayes", True)
                    sumarization = form_sumarization(sentences_to_keep)
                    predicted_sents = tf_idf_sumarization(file_article, category.test_files, method, params, n_grams)
                    scores = scorer.score(predicted_sents, sumarization)
                    count += 1
                    if count < 5:
                        print_file("Text:(train_files, categoria " + category.type + ' cu metoda ' + methods[method] + ' si folosind ' + _n_grams_[n_grams] + ")\n", file_article)

                        print_file("Summary:\n", file_summary)
                        print("\nPredicted summarisation:\n\n", predicted_sents)
                    if count == 5:
                        break
                        
                for (file_article, file_summary) in category.test_files:
                    sentences_to_keep = parse_document(file_summary, False, False, n_grams, "Naive Bayes", True)
                    sumarization = form_sumarization(sentences_to_keep)
                    predicted_sents = tf_idf_sumarization(file_article, category.test_files, method, params, n_grams)
                    scores = scorer.score(predicted_sents, sumarization)
                    count += 1
                    if count < 5:
                        print_file("Text:(test files, categoria " + category.type + ' cu metoda ' + methods[method] + ' si folosind ' + _n_grams_[n_grams] + ")\n", file_article)

                        print_file("Summary:\n", file_summary)
                        print("\nPredicted summarisation:\n\n", predicted_sents)
                        
                    if len(cantitative_analysis[category.type]) < 3:
                        with open(file_article, 'r') as file:
                            data = file.read()
                        cantitative_analysis[category.type] += [(data, predicted_sents, sumarization)]
                    if n_grams == UNIGRAMS:
                        if len(predicted_sents.split()) != 0:
                            precisions.append(bleu_score(predicted_sents, sumarization, n_grams))
                        else:
                            precisions.append(0)
                        recalls.append(scores['rouge1'].recall)
                    elif n_grams == BIGRAMS:
                        if len(predicted_sents.split()) != 0:
                            precisions.append(bleu_score(predicted_sents, sumarization, n_grams))
                        else:
                            precisions.append(0)
                        recalls.append(scores['rouge2'].recall)
                    else:
                        if len(predicted_sents.split()) != 0:
                            precisions.append(bleu_score(predicted_sents, sumarization, n_grams))
                        else:
                            precisions.append(0)
                        recalls.append(scores['rouge4'].recall)
                if n_grams == UNIGRAMS:
                    key = category.type + " unigrams"
                elif n_grams == BIGRAMS:
                    key = category.type + " bigrams"
                else:
                    key = category.type + " 4-grams"
                _precisions[key].append(100 * mean(precisions))
                _recalls[key].append(100 * mean(recalls))
                precisions_complete[category.type].append(100 * mean(precisions))
                recalls_complete[category.type].append(100 * mean(recalls))
                print('Precizia clasei ', category.type, 'cu metoda', methods[method],  
                    'si folosind', _n_grams_[n_grams], 'este:', 100 * mean(precisions), '%')
                print('Regasirea clasei ', category.type,'cu metoda', methods[method],
                      'si folosind', _n_grams_[n_grams], 'este:', 100 * mean(recalls), '%')
            print('====================================================================================================')

    return (_precisions, _recalls, precisions_complete, recalls_complete, cantitative_analysis)

In [ ]:
(__precisions, __recalls, __precisions_complete, __recalls_complete, cantitative_analysis) = evaluate_model_tf_idf()

In [ ]:
for _class in classes:
    for _tuple in cantitative_analysis[_class]:
        print(f'Text:\n{_tuple[0]}\nPredicted sumarization:\n{_tuple[1]}\nSumarization:\n{_tuple[2]}')
        print('======================================================================================================')

In [ ]:
def _autolabel(rects, ax):
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width() / 2.,
                0.5 * height,
                round(height, 5),
                rotation = 90,
                fontsize = 10,
                ha='center',
                va='bottom')

In [ ]:
def _plot_classification_category_res(category,
    precisions, recalls,
    methods, n_gram):
    fig, ax =  plt.subplots()
    X = range(len(methods))
    prec_rect = ax.bar(
        X,
        precisions,
        width = 0.25,
        label = "Precision")
    rec_rect = ax.bar(
        list(map(lambda x: x + 0.25, X)),
        recalls,
        width = 0.25,
        label = "Recall")
    ax.set_title(f"Documents from category {category} performing{n_gram}")
    ax.set_xlabel("Methods")
    ax.set_xticks(range(len(methods)))
    ax.set_xticklabels(
        methods,
        rotation = 60,
        ha = "right",
        rotation_mode = "anchor")
    ax.legend()
    _autolabel(prec_rect, ax)
    _autolabel(rec_rect, ax)

In [ ]:
def _plot_classification_category_res_complete(category,
    values,
    methods, _label):
    fig, ax =  plt.subplots()
    X = range(len(methods))
    values_rect = ax.bar(
        X,
        values,
        width = 0.25,
        label = _label)
    ax.set_title(f"Documents from category {category}")
    ax.set_xlabel(_label)
    ax.set_xticks(range(len(methods)))
    ax.set_xticklabels(
        methods,
        rotation = 60,
        ha = "right",
        rotation_mode = "anchor")
    ax.legend()
    _autolabel(values_rect, ax)

In [ ]:

def plot_graphics(precisions, recalls, methods):
    n_grams = [" unigrams", " bigrams", " 4-grams"]
    for n_gram in n_grams:
        for _class in classes:
            key = _class + n_gram
            _plot_classification_category_res(_class, precisions[key], recalls[key], methods, n_gram)

In [ ]:
def plot_graphics_complete(values, label):
    for _class in classes:
            _plot_classification_category_res_complete(_class, values[_class], methods_complete, label)

In [ ]:
from copy import deepcopy

In [ ]:
precisions_complete = {}
recalls_complete = {}
for key in _precisions_complete:
    precisions_complete[key] = deepcopy(_precisions_complete[key])
for key in __precisions_complete:
    precisions_complete[key] += __precisions_complete[key]
print(len(precisions_complete["business"]))
for key in _recalls_complete:
    recalls_complete[key] = deepcopy(_recalls_complete[key])
for key in __recalls_complete:
    recalls_complete[key] += __recalls_complete[key]

In [ ]:
plot_graphics(_precisions, _recalls, methods_naive_bayes)
plot_graphics(__precisions, __recalls, methods_tf_idf)
plot_graphics_complete(precisions_complete, "Precisions")
plot_graphics_complete(recalls_complete, "Recalls")